# Huge Tensor Bug - Data loss

## Requirements
I'm running this on the standard 0.7
[RAPIDS docker containers](https://hub.docker.com/r/rapidsai/rapidsai) and also
need the following `pip` dependencies installed:

In [26]:
!pip install torch==1.0.1

If you're running this on your local machine you should have most things installed

## CODE

### Imports

In [1]:
import torch

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

#### In order to demonstrate this bug we need a large tensor

In [4]:
features = torch.LongTensor(15000000, 45).random_(0, 2**22)
#features = torch.FloatTensor(15000000, 90).random_(-1, 1)

In [5]:
features[0]

tensor([2874842, 1426886, 2900003, 1340271, 1255904,   53916,  150306, 3135209,
        1566565, 3700984,  199050, 2326381,  436634,    1117, 1546400, 4001063,
        1472543, 1806464,  201497,  716067,   94419,  212859,  543911,  908213,
        4174394, 3005976, 2171144, 2967139,  813713, 1516383,  686261, 3008736,
        1182924, 3776220, 3716994, 3882399, 4064310, 2592183, 3436898, 1882412,
         546035, 3549929, 4055420, 1432250, 1471965])

In [6]:
features[14999999]

tensor([2167864, 3165248, 1245852, 2146404, 3948645,  265299,  844671, 3354802,
         590113, 2841125,  958101, 1508428, 3405411, 1697418,  301883, 3097217,
         620701,  818595, 3024319, 3623090, 1033113,  289799,  200632, 3951973,
        2293208, 1706245, 1716449, 3183350, 2641135, 1216988, 1998064, 2379344,
        4049610, 3027779, 1925729, 3419792, 2398803, 3161385, 2171724, 2212434,
        2186084, 3258908, 1923136, 3271952, 3799253])

Validate that the last element in the dataframe has data

In [ ]:
features2 = features.cuda()

In [ ]:
features2[0]

In [ ]:
features2[14999999]

In [ ]:
features3 = features2[torch.randperm(len(features2))]

In [ ]:
features3[0]

In [ ]:
features3[14999999]

Here's the boundary where this issue comes into play for this tensor.  All data after this point is 0.

In [ ]:
features3[11930460:11930470]